In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import math
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPool1D, Dropout, Flatten, Dense,Input
from keras.models import Model
from keras import optimizers
from keras.layers import merge
import matplotlib.pyplot as plt
#from sklearn.preprocessing import StandardScaler

def dh1(x):
	temp=np.zeros(x.shape)
	temp[:]=x[:]
	for i in range(len(x[:,-1])-2):
		x[i+1,-1]=int((temp[i+2,-1]-temp[i,-1])/2)
	return np.expand_dims(x[1:len(x)-1,-1], 1)
def standardd(X):
	Xnew=np.asarray(X,dtype='float')
	total=0
	u=X.sum()/len(X)
	for i in range(len(X)):
		total+=((X[i]-u)**2)
	o=math.sqrt(total/len(X))
	for i in range(len(X)):
		Xnew[i]=float((X[i]-u)/o)
	return Xnew
def processdata1(X_data):
  X_datanew=[]
  for i in range(X_data.shape[0]):
	  X=dh1(X_data[i])
	  X_datanew.append(X)
  X_datanew=np.asarray(X_datanew)
  for i in range(X_datanew.shape[0]):
	  X_datanew[i]=standardd(X_datanew[i])
  return X_datanew
def processdata2(X_data):
  for i in range(X_data.shape[0]):
    X_data[i]=standardd(X_data[i])
  return X_data

training=np.load('drive/My Drive/ML_tutorial/traininghigh.npz')
X_data=training['xdata']
y_true=training['ytrue']
y_onehot = np.zeros((len(y_true), 9))
y_onehot[np.arange(len(y_true)), y_true] = 1
X_train, X_test, y_train, y_test = train_test_split(X_data, y_onehot)

X_train1=processdata1(X_train)
X_train2=processdata2(X_train)
X_test1=processdata1(X_test)
X_test2=processdata2(X_test)

X_train1=X_train1.reshape(X_train1.shape[0],226,1)
X_train2=X_train2.reshape(X_train2.shape[0],228,1)

#first layer
input_shape=(226,1)
model1=Input(input_shape)
conv1=Conv1D(16,2,activation='relu')(model1)
maxp1=MaxPool1D(2)(conv1)
conv2=Conv1D(32,2,activation='relu')(maxp1)
maxp2=MaxPool1D(2)(conv2)
conv3=Conv1D(32,2,activation='relu')(maxp2)
maxp3=MaxPool1D(2)(conv3)
conv4=Conv1D(64,2,activation='relu')(maxp3)
maxp4=MaxPool1D(2)(conv4)
flat1=Flatten()(maxp4)
#second layer
input_shape2=(228,1)
model2=Input(input_shape2)
conv12=Conv1D(16,2,activation='relu')(model2)
maxp12=MaxPool1D(2)(conv12)
conv22=Conv1D(32,2,activation='relu')(maxp12)
maxp22=MaxPool1D(2)(conv22)
conv32=Conv1D(32,2,activation='relu')(maxp22)
maxp32=MaxPool1D(2)(conv32)
conv42=Conv1D(64,2,activation='relu')(maxp32)
maxp42=MaxPool1D(2)(conv42)
print(type(maxp42))
flat2=Flatten()(maxp42)

#merge layers
merge=merge.concatenate([flat1,flat2])
dense1=Dense(128,activation='relu')(merge)
output=Dense(9,activation='softmax')(dense1)

model=Model(inputs=[model1,model2],outputs=output)
adam = optimizers.Adam(0.0001, 0.9, 0.99)
model.compile(adam, 'categorical_crossentropy', ['accuracy'])
model.summary()
#from keras.utils import plot_model
#plot_model(model, to_file='drive/My Drive/ML_tutorial/model.png', show_shapes=True, show_layer_names=True)
history = model.fit([X_train1,X_train2], y_train, 64, 2000, 1,validation_data=([X_test1,X_test2],y_test))
with open("drive/My Drive/ML_tutorial/Model/modelhigh2526.json","w") as model_file:
	model_file.write(model.to_json())
model.save("drive/My Drive/ML_tutorial/Weight/modelhigh2526.h5")

(1513, 228, 1)
(505, 228, 1)
<class 'tensorflow.python.framework.ops.Tensor'>
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 226, 1)       0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 228, 1)       0                                            
__________________________________________________________________________________________________
conv1d_49 (Conv1D)              (None, 225, 16)      48          input_13[0][0]                   
__________________________________________________________________________________________________
conv1d_53 (Conv1D)              (None, 227, 16)      48          input_14[0][0]                   
______________

'history = model.fit([X_train1,X_train2], y_train, 64, 2000, 1,validation_data=([X_test1,X_test2],y_test))\nwith open("drive/My Drive/ML_tutorial/Model/modelhigh2526.json","w") as model_file:\n\tmodel_file.write(model.to_json())\nmodel.save("drive/My Drive/ML_tutorial/Weight/modelhigh2526.h5")'